# Paths with key players
Analysis of key players that are involved in HemeMap and pathway databases (KEGG, Reactome and WikiPathways converted into BEL using PathMe python package).


In [1]:
import os
import sys
import time
import pybel
import logging
import pybel_tools

from itertools import chain
import pickle
from pybel_jupyter import to_jupyter # Visualize network
from pybel.struct.mutation import collapse_all_variants, collapse_to_genes
import matplotlib.pyplot as plt
from matplotlib_venn import venn2, venn3, venn2_unweighted, venn3_unweighted

from bio2bel_kegg import Manager as KeggManager
from bio2bel_reactome import Manager as ReactomeManager
from bio2bel_wikipathways import Manager as WikipathwaysManager

from networkx.algorithms import shortest_path, all_simple_paths # From a(MESH:Heme) to path(MESH:Thrombosis)
import networkx as nx

from pybel.constants import ANNOTATIONS, RELATION
from collections import Counter, defaultdict

import heme_knowledge

In [2]:
sys.version

'3.7.3 (default, Mar 27 2019, 17:13:21) [MSC v.1915 64 bit (AMD64)]'

In [3]:
time.asctime()

'Fri Jun 21 08:58:51 2019'

In [4]:
pybel.get_version()

'0.13.2'

In [5]:
logging.basicConfig(level=20)

In [6]:
merged_network = heme_knowledge.get_graph()

In [7]:
pybel.struct.remove_associations(merged_network)

In [8]:
collapse_to_genes(merged_network)

In [9]:
#pybel.struct.mutation.remove_isolated_list_abundances(merged_network)

In [10]:
#pybel.struct.mutation.remove_biological_processes(merged_network)

In [11]:
#to_jupyter(merged_network)

#### Create a subgraph of path from heme to target node to find the key players in paths

In [12]:
def bel_to_simple_directed(graph):
    """BELGraph to directed NetworkX graph reducing redundant edges."""
    directed_graph = nx.DiGraph()
    
    for sub, obj in graph.edges():
        directed_graph.add_edge(sub, obj)

    return directed_graph

merged_network_directed = bel_to_simple_directed(merged_network)

#gives the paths from heme-thrombosis
for node in merged_network_directed:
        if 'name' in node and node.name == 'heme':
            source_node = node
        
        elif 'name' in node and node.name == 'Inflammation':
            target_node = node
            
#         if not source_node or not target_node:
#             raise ValueError('One of the nodes was not found')

data = nx.all_simple_paths(merged_network_directed, source_node, target_node, cutoff= 6)

data = list(data)

#Gives all the nodes present in the path
all_nodes_in_paths = set(chain.from_iterable(data))

print(f"Number of simple paths for {source_node.name}-{target_node.name}: {len(data)}")

#Gives those which involoves the key_players in paths leading from heme-endpoints (thrombosis, inflammation, hemolysis)
def via_paths(paths, node_name):
    
    via_key_players_list = []
        
    for path in paths:
        for node in path:
            if 'name' in node and node.name == node_name:
                via_key_players_list.append(path)
                
    return via_key_players_list

key_nodes = via_paths(data, 'TLR4')

#flattens the list of list for graph generation
def flatten_list(list_of_list):
    
    key_nodes_flat = list()
    
    for sublist in list_of_list:
        for element in sublist:
            
            key_nodes_flat.append(element)
            
    return key_nodes_flat

graph_nodes_key_players = flatten_list(key_nodes)

# sub_graph generation from heme_network 
graph = pybel.struct.union([
    merged_network.subgraph(graph_nodes_key_players)
    
])

Number of simple paths for heme-Inflammation: 70488


In [13]:
to_jupyter(graph)

<IPython.core.display.Javascript object>

In [14]:
# pickle_out = open('thrombosis_nodes_with_pathways.pickle','wb')
# pickle.dump(graph_nodes_key_players, pickle_out)
# pickle_out.close()

In [15]:
# thrombosis_dict = pickle.load(open('thrombosis_nodes_with_pathways.pickle', 'rb'))

In [16]:
#key_nodes

In [17]:
# test = set(graph_nodes_key_players)
# list_test = list(test)

In [18]:
# for node in list_test:
#     if 'namespace' in node and node.namespace == 'MGI':
#         no_MGI = list_test.remove(node)

In [19]:
# print(no_MGI)

In [20]:
print(graph_nodes_key_players)

[<BEL a(CHEBI:heme)>, <BEL a(MESH:"Reactive Oxygen Species")>, <BEL bp(GO:"inflammatory response")>, <BEL g(HGNC:TLR4)>, <BEL a(MESH:"Mitogen-Activated Protein Kinases")>, <BEL g(HGNC:TNF)>, <BEL path(MESH:Inflammation)>, <BEL a(CHEBI:heme)>, <BEL a(MESH:"Reactive Oxygen Species")>, <BEL bp(GO:"inflammatory response")>, <BEL g(HGNC:TLR4)>, <BEL g(HGNC:TNF)>, <BEL path(MESH:Inflammation)>, <BEL a(CHEBI:heme)>, <BEL a(MESH:"Reactive Oxygen Species")>, <BEL bp(GO:"inflammatory response")>, <BEL g(HGNC:TLR4)>, <BEL g(HGNC:MYD88)>, <BEL g(HGNC:TNF)>, <BEL path(MESH:Inflammation)>, <BEL a(CHEBI:heme)>, <BEL a(MESH:"Reactive Oxygen Species")>, <BEL bp(GO:"inflammatory response")>, <BEL g(HGNC:TLR4)>, <BEL g(HGNC:NFKB1)>, <BEL g(HGNC:TNF)>, <BEL path(MESH:Inflammation)>, <BEL a(CHEBI:heme)>, <BEL a(MESH:"Reactive Oxygen Species")>, <BEL g(MGI:Hpx)>, <BEL bp(GO:"inflammatory response")>, <BEL g(HGNC:TLR4)>, <BEL g(HGNC:TNF)>, <BEL path(MESH:Inflammation)>, <BEL a(CHEBI:heme)>, <BEL bp(GO:"infla